In [7]:
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import pandas as pd
from splinter import Browser
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [8]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - There is no [mac64] chromedriver for browser 89.0.4389 in cache
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/bailey/.wdm/drivers/chromedriver/mac64/89.0.4389.23]


In [9]:
news_url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(news_url)

In [10]:
html = browser.html

# Parse 
soup = BeautifulSoup(html, 'html.parser')

#Select latest 
browser.find_by_id('date').first.click()
browser.find_option_by_text('Latest').first.click()

#Select All
browser.find_by_id('categories').first.click()
browser.find_option_by_text('All Categories').first.click()
browser.find_by_id('categories').first.click()
time.sleep(0)

In [12]:
#Collect the latest News Title
results=soup.find('li', class_="slide")

# Pase the first title from the results 
news_title =results.find('div',class_='content_title').text.strip()

# Parse out the first paragraph associatd with the tile from results
news_p=results.find('div',class_='article_teaser_body').text.strip()



In [13]:
#Use splinter to browse through page
images_url="https://www.jpl.nasa.gov/images?search=&category=Mars"
browser.visit(images_url)
time.sleep(1)

In [14]:
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [15]:
browser.find_by_id('filter_Mars').click()
time.sleep(0.5)
featured_image_url=browser.find_by_css('.BaseImage')['data-src']

In [17]:
facts_url="https://space-facts.com/mars/"
browser.visit(facts_url)
facts_tables = pd.read_html(facts_url)
# facts_tables

mars_0_df=facts_tables[0]
# mars_0_df

column_names={
    0: "Description",
    1: "Mars Fact"
}

mars_1_df=mars_0_df.rename(columns=column_names)
# mars_1_df

mars_df=mars_1_df.set_index("Description")
# mars_df

mars_facts_html=mars_df.to_html()


In [18]:
hemispheres_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)


In [19]:
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [20]:
results = soup.find_all('div', class_='item')
results

hemisphere_image_urls=[]
count=0

for each_result in results: 
    
    title=each_result.find('h3').text

    
    browser.find_by_tag("h3")[count].click()
    time.sleep(0.5)
    
    count=count+1
    
    html = browser.html

    img_url=browser.find_by_text('Sample')['href']
    
    hemisphere_dict={
        "title": title,
        "img_url":img_url
    }
    
    hemisphere_image_urls.append(hemisphere_dict)
    
    time.sleep(0.5)
    
    browser.visit(hemispheres_url)
    

In [21]:
browser.quit()


In [23]:
mars_dict={
    "Title":news_title,
    "Paragraph":news_p,
    "FeaturedImage": featured_image_url,
    "MarsFacts": mars_facts_html,
    "HemisphereImages": hemisphere_image_urls
}

print(mars_dict)

{'Title': 'NASA Ingenuity Mars Helicopter Prepares for First Flight', 'Paragraph': 'Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.', 'FeaturedImage': 'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24508.2e16d0ba.fill-400x400-c50.jpg', 'MarsFacts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars Fact</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit P